#### Library

In [1]:
import importlib
import Training_utils
import torchvision
from torchvision import transforms
import tb_utils
importlib.reload(tb_utils)
from tb_utils import *
importlib.reload(Training_utils)
from Training_utils import *
from Image_Process_utils import *
import model_utils
importlib.reload(model_utils)
from model_utils import *
from Dataset_utils import *
if not os.path.exists('./checkpoints'):
    os.mkdir('./checkpoints')
import arg_parser
importlib.reload(arg_parser) 
from arg_parser import common_arg_parser

In [2]:
global t_step
global v_step
global test_step
global micro_step
global v_micro
t_step = v_step = test_step = micro_step = v_micro = 0

In [3]:
arg_parser =common_arg_parser()
args, unknown_args = arg_parser.parse_known_args()

In [4]:
batch_size =12
my_transforms = transforms.Compose([
    #transforms.Normalize((0.5,), (0.5,)),
    Rescale(256),
    Tensorize()
    ])
img_dir = './Acu_Dataset/sotack/org'
json_file = './Acu_Dataset/sotack/sotack_info.json'

sotack_dataset = HandDataSet(json_file, img_dir, transform = my_transforms, train=True) # pytorch 로 변환

set()


In [5]:
with open(json_file) as f:
    json_data = json.load(f)

In [6]:
test_set_size = int(len(sotack_dataset) * 0.1)
train_set_size = len(sotack_dataset) - test_set_size
train_set, test_set = torch.utils.data.random_split(sotack_dataset, [train_set_size, test_set_size])

valid_set_size = int((train_set_size)*0.1) 
train_set_size = train_set_size - valid_set_size

train_set, valid_set  = torch.utils.data.random_split(train_set, [train_set_size, valid_set_size])

In [7]:
train_loader = DataLoader(train_set, batch_size = 32, shuffle = True, num_workers=4)
valid_loader = DataLoader(valid_set, batch_size = 32, shuffle = True, num_workers=4)
test_loader = DataLoader(test_set, batch_size = 32, shuffle = True, num_workers=4)

In [8]:
model_name = args.model
model = create_model(model_name)
summary(model.to('cuda'), input_size = (3, 700, 700))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 350, 350]           9,408
       BatchNorm2d-2         [-1, 64, 350, 350]             128
              ReLU-3         [-1, 64, 350, 350]               0
         MaxPool2d-4         [-1, 64, 175, 175]               0
            Conv2d-5         [-1, 64, 175, 175]          36,864
       BatchNorm2d-6         [-1, 64, 175, 175]             128
              ReLU-7         [-1, 64, 175, 175]               0
            Conv2d-8         [-1, 64, 175, 175]          36,864
       BatchNorm2d-9         [-1, 64, 175, 175]             128
             ReLU-10         [-1, 64, 175, 175]               0
       BasicBlock-11         [-1, 64, 175, 175]               0
           Conv2d-12         [-1, 64, 175, 175]          36,864
      BatchNorm2d-13         [-1, 64, 175, 175]             128
             ReLU-14         [-1, 64, 1

In [9]:
best_val = 100000000
learning_rate = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate, betas = (0.5, 0.999))
#torch.optim.SGD(model.parameters(), args.lr,momentum=args.momentum,weight_decay=args.weight_decay)
#momentum = 0.9 
#dataloader = train_loader
scheduler = StepLR(optimizer, step_size = 1, gamma = 0.5)
class_loss = class_loss()
coord_loss = coord_loss()

In [16]:
today = str(datetime.datetime.now().today())
time_info = today[5:7] +today[8:10] + '_' + today[11:13] + today[14:16]
args.prefix = 'test' + time_info
args.epochs = 50
writer = SummaryWriter('runs/'+args.prefix)

In [11]:
# for creating 
test_imlist, target_imlist =  gen_test_img_list()

In [17]:
#step1 = 0
for epoch in range(args.start_epoch, args.epochs):
    tr_loss, micro_step, t_step =  train(model, train_loader, optimizer, coord_loss, class_loss, epoch, writer, micro_step, t_step)
    val_loss, v_micro, v_step = validate(model, valid_loader, coord_loss, class_loss, epoch, writer, v_micro, v_step)
    test_step = test(test_imlist, target_imlist, model, writer, 'test_image', test_step)
    writer.flush()
    
    val_loss
    is_best = sum(val_loss[0:2]) > best_val
    best_val = min( sum(val_loss[0:2]), best_val )
    
    save_checkpoint({
        'epoch' : epoch,
        'model' : args.model, 
        'state_dict' : model.state_dict(),
        'best_val' : best_val, 
        'optimizer' : optimizer.state_dict(),
        'scheduler' : scheduler.state_dict()
    }, is_best, args.prefix)
    
    
    
#    scheduler.step()
    
    

Epoch[1] Batch 10/128  Time 0.739s Data 0.574s || LOSS: CLS 0.069 X 577.18 Y 283.41 tot 860.655 | Current: 18:45:12
Epoch[1] Batch 20/128  Time 0.464s Data 0.302s || LOSS: CLS 0.068 X 632.74 Y 328.19 tot 960.996 | Current: 18:45:14
Epoch[1] Batch 30/128  Time 0.366s Data 0.206s || LOSS: CLS 0.073 X 571.85 Y 290.99 tot 862.908 | Current: 18:45:15
Epoch[1] Batch 40/128  Time 0.316s Data 0.157s || LOSS: CLS 0.068 X 520.15 Y 266.08 tot 786.296 | Current: 18:45:17
Epoch[1] Batch 50/128  Time 0.285s Data 0.127s || LOSS: CLS 0.065 X 548.89 Y 267.43 tot 816.386 | Current: 18:45:18
Epoch[1] Batch 60/128  Time 0.265s Data 0.107s || LOSS: CLS 0.063 X 541.87 Y 256.34 tot 798.275 | Current: 18:45:20
Epoch[1] Batch 70/128  Time 0.251s Data 0.092s || LOSS: CLS 0.061 X 500.13 Y 238.84 tot 739.030 | Current: 18:45:22
Epoch[1] Batch 80/128  Time 0.240s Data 0.081s || LOSS: CLS 0.060 X 474.20 Y 227.69 tot 701.957 | Current: 18:45:23
Epoch[1] Batch 90/128  Time 0.231s Data 0.073s || LOSS: CLS 0.059 X 483.

Epoch[6] Batch 60/128  Time 0.273s Data 0.109s || LOSS: CLS 0.018 X 413.29 Y 198.70 tot 612.010 | Current: 18:48:39
Epoch[6] Batch 70/128  Time 0.257s Data 0.094s || LOSS: CLS 0.019 X 412.33 Y 192.91 tot 605.263 | Current: 18:48:40
Epoch[6] Batch 80/128  Time 0.246s Data 0.083s || LOSS: CLS 0.018 X 406.22 Y 195.44 tot 601.687 | Current: 18:48:42
Epoch[6] Batch 90/128  Time 0.237s Data 0.074s || LOSS: CLS 0.017 X 395.15 Y 193.37 tot 588.541 | Current: 18:48:44
Epoch[6] Batch 100/128  Time 0.231s Data 0.067s || LOSS: CLS 0.016 X 374.91 Y 181.18 tot 556.107 | Current: 18:48:45
Epoch[6] Batch 110/128  Time 0.225s Data 0.062s || LOSS: CLS 0.015 X 392.22 Y 184.62 tot 576.860 | Current: 18:48:47
Epoch[6] Batch 120/128  Time 0.221s Data 0.057s || LOSS: CLS 0.015 X 376.23 Y 173.26 tot 549.502 | Current: 18:48:49
Epoch[6] Batch 10/15  Time 0.820s || LOSS: CLS 0.003 X 362.35 Y 178.64 tot 540.998 | Current: 18:48:59
writing test img with step 7
Epoch[7] Batch 10/128  Time 0.744s Data 0.580s || LOS

Epoch[11] Batch 110/128  Time 0.221s Data 0.062s || LOSS: CLS 0.331 X 1151.24 Y 315.48 tot 1467.048 | Current: 18:52:06
Epoch[11] Batch 120/128  Time 0.216s Data 0.057s || LOSS: CLS 0.321 X 1108.27 Y 307.90 tot 1416.487 | Current: 18:52:08
Epoch[11] Batch 10/15  Time 0.822s || LOSS: CLS 1.900 X 8553.47 Y 2941.41 tot 11496.779 | Current: 18:52:18
writing test img with step 12
Epoch[12] Batch 10/128  Time 0.746s Data 0.583s || LOSS: CLS 0.136 X 582.63 Y 171.65 tot 754.422 | Current: 18:52:30
Epoch[12] Batch 20/128  Time 0.467s Data 0.307s || LOSS: CLS 0.111 X 696.58 Y 288.41 tot 985.102 | Current: 18:52:31
Epoch[12] Batch 30/128  Time 0.367s Data 0.210s || LOSS: CLS 0.091 X 560.94 Y 248.69 tot 809.722 | Current: 18:52:33
Epoch[12] Batch 40/128  Time 0.317s Data 0.159s || LOSS: CLS 0.080 X 462.18 Y 209.64 tot 671.896 | Current: 18:52:34
Epoch[12] Batch 50/128  Time 0.286s Data 0.129s || LOSS: CLS 0.079 X 435.94 Y 197.52 tot 633.547 | Current: 18:52:36
Epoch[12] Batch 60/128  Time 0.266s D

KeyboardInterrupt: 

In [ ]:
args.resume = './checkpoints/Hello_checkpoint.pth.tar'

if args.resume:
    if os.path.isfile(args.resume):
        print("=> loading checkpoint '{}'".format(args.resume))
        checkpoint = torch.load(args.resume)
        args.start_epoch = checkpoint['epoch']
        best_val = checkpoint['best_val']
        model.load_state_dict(checkpoint['state_dict'])
        if 'optimizer' in checkpoint:
            optimizer.load_state_dict(checkpoint['optimizer'])
        print("=> loaded checkpoint '{}' (epoch {})"
              .format(args.resume, checkpoint['epoch']))
    else:
        print("=> no checkpoint found at '{}'".format(args.resume))

#### test 체킹할때

In [ ]:
_, out2 = model(test_imlist[0].unsqueeze(0).to('cuda'))

In [ ]:
out2.shape

In [ ]:
model(test_imlist[0].unsqueeze(0))

In [ ]:
test(test_imlist, target_imlist, model, writer, 'test_image', step = 1)

In [ ]:
d1['image'].shape

#### tb 체킹

In [ ]:
d1 = iter(train_loader).__next__()

In [ ]:
p1, p2 = model(d1['image'].to('cuda'))
t1, t2, t3 = d1['image'], d1['target'], d1['label']
img = np.transpose( to_numpy(t1[0]), (1,2,0))
predicted =  to_numpy(p2[0])
target = to_numpy(t2[0][1:])
label = t3[0]

In [ ]:
json_data['sotack_0002983']

In [ ]:
monitor_change(img, target, predicted, label )

In [ ]:
img_list = gen_train_monitor_plot(imgs, out2, x_ind = 0, y_ind = 1,target =  targets)
imgs_grid = torchvision.utils.make_grid(img_list, normalize = True)
writer = SummaryWriter(f'runs/hands')
writer.add_image('imgs', imgs_grid)
writer.flush()